In [1]:
import os
os.environ['HF_HOME'] = 'e:/.cache/huggingface'

from pathlib import Path
import shutil
from dataclasses import dataclass

from tqdm.auto import tqdm
import gradio as gr
import datasets
from datasets import load_dataset

sample_text = Path('datasets/sample_text.txt').read_text()

In [ ]:
from punctuators.models import PunctCapSegModelONNX

m: PunctCapSegModelONNX = PunctCapSegModelONNX.from_pretrained(
    '1-800-BAD-CODE/xlm-roberta_punctuation_fullstop_truecase',
)

punct_text_sentences = m.infer(texts=[sample_text], apply_sbd=True)[0]
print(
    len(punct_text_sentences),
    len(' '.join(punct_text_sentences)),
    max([len(s) for s in punct_text_sentences])
)

Path('datasets/punct_text_sentences.txt').write_text('\n'.join(punct_text_sentences))

In [3]:
punct_text_sentences = Path('datasets/punct_text_sentences.txt').read_text().split('\n')

In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50-many-to-many-mmt') # .cuda()
tokenizer = MBart50TokenizerFast.from_pretrained('facebook/mbart-large-50-many-to-many-mmt')

: 

In [15]:
chunks = ['']
for sentence in punct_text_sentences:
    new_text = chunks[-1] + ' ' + sentence
    if len(tokenizer(new_text)['input_ids']) < 1024:
        chunks[-1] = new_text
    else:
        chunks.append(sentence)
print(
    len(chunks),
    len(' '.join(chunks)),
    max([len(s) for s in chunks])
)

12 42995 3829


In [ ]:
tokenizer.src_lang = 'ru_RU'
generated_tokens = model.generate(
    **tokenizer(' '.join(punct_text_sentences), return_tensors='pt'),
    forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"],
    max_new_tokens=99999,
)
print(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))

In [14]:
tokenizer('a fewfef efer fer ')['input_ids']

[250021, 10, 10846, 2242, 420, 28, 2875, 1592, 2]